# Scout Dataset Tracking Evaluation Examples

This notebook demonstrates two approaches for computing tracking metrics on the Scout dataset:

1. **3D Tracking Metrics**: Using `compute_mot_metric` for evaluation in 3D world coordinates
2. **Monocular Tracking Metrics**: Using `MOTMetricEvaluator` for standard MOT evaluation

Both methods work by:
- Loading a sequence of ground truth annotations
- Simulating predictions by randomly modifying the ground truth
- Computing tracking metrics between GT and predictions

In [ ]:
import numpy as np
import numpy as np
if not hasattr(np, 'float'):
    np.float = float
if not hasattr(np, 'int'):
    np.int = int
if not hasattr(np, 'bool'):
    np.bool = bool
if not hasattr(np, 'object'):
    np.object = object
if not hasattr(np, 'str'):
    np.str = str
    
import pandas as pd
import torch
import random
from pathlib import Path
import motmetrics as mm

from src.loading import Individual
from src.metric import (MOTMetricEvaluator, compute_mot_metric, convert_scout_to_tensors, 
                        scout_to_dataframe, simulate_predictions, validate_unique_ids)

In [ ]:
# Configuration
rootdir = '/path/to/scout/'
sequence_id = 1
camera_id = 'cam_2'

# Set random seed for reproducible results
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

In [ ]:
## Load Scout Dataset
indiv = Individual(rootdir, sequence=sequence_id)
print(f"Loaded sequence {sequence_id} from {rootdir}")

In [ ]:
## Load Ground Truth Sequence
# Load annotations for multiple frames to create a sequence
start_frame = 80
end_frame = 90
frame_range = range(start_frame, end_frame + 1)

gt_sequence = {}
for frame_id in frame_range:
    try:
        ann = indiv.retrieve(frame_id, camera_id)
        if ann:  # Only add frames with annotations
            gt_sequence[frame_id] = ann
    except Exception as e:
        print(f"Could not load frame {frame_id}: {e}")
        continue

print(f"Loaded {len(gt_sequence)} frames with annotations")
print(f"Frame range: {min(gt_sequence.keys())} to {max(gt_sequence.keys())}")

# Show first frame as example
if gt_sequence:
    first_frame = min(gt_sequence.keys())
    print(f"\nExample annotation for frame {first_frame}:")
    print(f"Number of detections: {len(gt_sequence[first_frame])}")
    print(f"First detection: {gt_sequence[first_frame][0]}")

In [ ]:
## Create Simulated Predictions
# All simulation functions are now imported from src.metric

# Generate predictions
pred_sequence = simulate_predictions(gt_sequence)

# Validate that predictions have unique IDs per frame
print("Validating unique IDs in predictions...")
if validate_unique_ids(pred_sequence):
    print("✓ All track IDs are unique within each frame")
else:
    print("✗ Found duplicate track IDs - regenerating predictions...")
    pred_sequence = simulate_predictions(gt_sequence)

print("\nGround Truth vs Predictions Summary:")
for frame_id in sorted(gt_sequence.keys()):
    gt_count = len(gt_sequence[frame_id])
    pred_count = len(pred_sequence.get(frame_id, []))
    print(f"Frame {frame_id}: GT={gt_count}, Pred={pred_count}")

In [ ]:
# ================================================================================================
# USE CASE 1: 3D Tracking Metrics using compute_mot_metric
# ================================================================================================

## Convert Scout annotations to the format expected by compute_mot_metric
# scout_to_dataframe function is now imported from src.metric

# Convert ground truth and predictions to dataframe format
print("Converting data to dataframe format for 3D tracking evaluation...")

# Using world coordinates for 3D evaluation
gt_df_3d = scout_to_dataframe(gt_sequence, use_world_coords=True)
pred_df_3d = scout_to_dataframe(pred_sequence, use_world_coords=True)

print(f"Ground Truth DataFrame shape: {gt_df_3d.shape}")
print(f"Predictions DataFrame shape: {pred_df_3d.shape}")
print(f"\nGround Truth DataFrame head:")
print(gt_df_3d.head())
print(f"\nPredictions DataFrame head:")
print(pred_df_3d.head())


In [ ]:
## Compute 3D Tracking Metrics using compute_mot_metric

# Define distance threshold for matching in meters (world coordinates)
distance_threshold_3d = 1.0  # 1 meter threshold for 3D matching

# Count ground truth objects for metrics computation
nb_gt = len(gt_df_3d)

print(f"Computing 3D tracking metrics with distance threshold: {distance_threshold_3d}m")
print(f"Number of ground truth detections: {nb_gt}")

# Compute metrics
if nb_gt > 0:
    metrics_3d = compute_mot_metric(gt_df_3d, pred_df_3d, distance_threshold_3d, nb_gt)
    
    if metrics_3d:
        print("\n=== 3D Tracking Metrics Results ===")
        print(f"MOTA (Multiple Object Tracking Accuracy): {metrics_3d.get('mota', 'N/A'):.3f}")
        print(f"MOTP (Multiple Object Tracking Precision): {metrics_3d.get('motp', 'N/A'):.3f}")
        print(f"IDF1 (ID F1 Score): {metrics_3d.get('idf1', 'N/A'):.3f}")
        print(f"Number of Switches: {metrics_3d.get('num_switches', 'N/A')}")
        print(f"Number of Fragmentations: {metrics_3d.get('num_fragmentations', 'N/A')}")
        print(f"Number of False Positives: {metrics_3d.get('num_false_positives', 'N/A')}")
        print(f"Number of Misses: {metrics_3d.get('num_misses', 'N/A')}")
        print(f"Precision: {metrics_3d.get('precision', 'N/A'):.3f}")
        print(f"Recall: {metrics_3d.get('recall', 'N/A'):.3f}")
        
        print(f"\nFull metrics dictionary:")
        for key, value in metrics_3d.items():
            print(f"  {key}: {value}")
    else:
        print("Failed to compute metrics")
else:
    print("No ground truth data available for metrics computation")

In [ ]:
# ================================================================================================
# USE CASE 2: Monocular Tracking Metrics using MOTMetricEvaluator  
# ================================================================================================

## Convert Scout data to format expected by MOTMetricEvaluator

print("Converting data for MOTMetricEvaluator...")

# Convert data using the cleaner helper function
gt_dict, pred_bboxes, pred_world_points, pred_timestamps, pred_ids = convert_scout_to_tensors(gt_sequence, pred_sequence)

print(f"Ground truth dictionary has {len(gt_dict)} timestamps")
print(f"Prediction tensors: bboxes {pred_bboxes.shape}, world_points {pred_world_points.shape}")
print(f"Prediction IDs: {len(pred_ids)} track assignments")


In [ ]:
## Run MOTMetricEvaluator

# trackeval library might conflict with recent data type updates of numpy. Replace np.float with float directly in trackeval code should fix it.


# Initialize the evaluator
evaluator = MOTMetricEvaluator(interpolate_missing_detections=False)

# Save sequence data  
print("Saving sequence data for MOT evaluation...")
gt_world_data, pred_world_data = evaluator.save_sequence_data(
    pred_bboxes=pred_bboxes,
    pred_world_points=pred_world_points,
    pred_timestamps=pred_timestamps,
    pred_ids=pred_ids, 
    gt_dict=gt_dict,
    dset_name="SCOUT",
    sequence="test_sequence"
)

print("Computing MOT metrics...")

# Compute metrics using trackeval
sequence_metrics = evaluator.compute_metrics("SCOUT")

print("\n=== MOT Evaluation Results (using trackeval) ===")

# Display key metrics
key_metrics = ['HOTA', 'DetA', 'AssA', 'MOTA', 'MOTP', 'IDF1', 'IDSW']

for metric in key_metrics:
    combined_key = f'combined_{metric}'
    if combined_key in sequence_metrics:
        print(f"{metric}: {sequence_metrics[combined_key]:.3f}")

print(f"\nFull metrics available:")
for key, value in sequence_metrics.items():
    print(f"  {key}: {value}")
        

